<h1>Project -  Stock Market Prediction</h1>


Question 1: Pre-processing &  data exploration

In [1]:
import nltk
nltk.download()
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import sklearn.datasets as sk
import matplotlib.pyplot as plt

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
stopWords=(stopwords.words('english'))

In [3]:
def cleanText(text):
    text = str(text)
    if text.startswith("b\"") or text.startswith("b\'"):
        text = text[1:]
    text = text.replace("\"", "").replace("\'", "")
    # lower case
    text = text.lower()
    # tokenize
    filtered_sentence = word_tokenize(text)
    # # remove stop words
    filtered_sentence = [w for w in filtered_sentence if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    # stemming
    ps = PorterStemmer()
    for i in range(len(filtered_sentence)-1):
        try:
            filtered_sentence[i] = ps.stem(filtered_sentence[i])
        except Exception as inst:
            filtered_sentence[i]=filtered_sentence[i]
    # remove stop words after stem
    filtered_sentence = [w for w in filtered_sentence if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    text= " ".join(filtered_sentence)
    return text.strip()

In [4]:
combined_news_djia_df = pd.read_csv( "C:\\Users\\nogahm\\PycharmProjects\\Project-Stock-Market-Prediction\\Data\\Combined_News_DJIA.csv")

In [6]:
headlines_columns = combined_news_djia_df.columns[range(2, 27)]
combined_news_djia_df[headlines_columns] = combined_news_djia_df[headlines_columns].applymap(cleanText)

In [7]:
samples_per_class = []
for c in [0, 1]:
    samples_per_class.append([c, combined_news_djia_df["Label"].value_counts()[c]])
pd.DataFrame(samples_per_class, columns=['Class', '# of Samples'])

,Class,# of Samples
0,0,924
1,1,1065


In [9]:
n = 10
for c in [0, 1]:
    print ('Class:', c)
    class_freq = pd.Series()
    class_df = combined_news_djia_df.loc[combined_news_djia_df["Label"] == c]
    for headline_column in combined_news_djia_df[headlines_columns]:
        class_freq = class_freq.append(pd.Series(" ".join(class_df[headline_column]).split()).value_counts()[:n])
    class_freq = class_freq.groupby(by=class_freq.index).sum().sort_values(ascending=False,axis=0)
    print(class_freq.to_frame('freq'))


Class: 0
         freq
ha       2190
say      1591
us       1287
wa       1153
kill     1102
world     971
new       914
year      833
china     563
govern    531
israel    490
polic     310
peopl     308
hi        275
russia    227
countri   196
attack    160
war        44
u.s.       42
report     41
first      41
thi        40
iran       39
Class: 1


         freq
ha       2495
say      1860
us       1471
kill     1363
wa       1190
new       994
world     926
china     895
year      829
israel    719
govern    527
polic     291
peopl     238
attack    192
hi        187
russia    149
war       138
countri   137
iran       86
use        58
ban        56
drug       55
protest    53
call       49
state      46
one        42
report     40


Question 2: machine learning classifier

In [10]:
from sklearn.model_selection import train_test_split
# x = combined_news_djia_df[combined_news_djia_df.columns[2:4]]
x=combined_news_djia_df.drop(['Label'], axis=1)
y = combined_news_djia_df[["Label"]]
print(x)

In [11]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.3, random_state=0)

In [12]:
xTrain = xTrain.apply(lambda x: " ".join(x), axis=1)
xTest = xTest.apply(lambda x: " ".join(x), axis=1)
xTrain=xTrain.tolist()
xTest=xTest.tolist()
yTrain=yTrain['Label'].tolist()
yTest=yTest['Label'].tolist()

In [13]:
import numpy as np
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron


The first model we will use is SVM (not Deep Learning, aka Keras)

In [14]:
def classify(extractM,machineLearningM, parameters): 
            pipeline = Pipeline([('vect', extractM), ('clf', machineLearningM)])
            gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)
            gs_clf = gs_clf.fit(xTrain, yTrain)
            prediction = gs_clf.predict(xTest)
            accuracy = metrics.accuracy_score(yTest, prediction)
            # print(accuracy)
            return [accuracy, gs_clf.best_params_]

In [15]:
results=[]
parameters = {'vect__max_df': (0.3,0.4,0.5,0.6,0.7,0.75, 1.0),'clf__alpha': (0.0001, 0.01,1.0)}
#TF-IDF & SVM
ans = classify(TfidfVectorizer(sublinear_tf=True,stop_words='english'),SGDClassifier() ,parameters)
accuracy = ans[0]
params = ans[1]
results.append(['TF-IDF' , 'SVM', params,accuracy])

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


...

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


The second model we will use is Neural Network (Deep Learning)

In [16]:
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


ImportError: DLL load failed: The specified procedure could not be found.

In [1]:


# design network
model = Sequential()
model.add(LSTM(50, input_shape=(xTrain.shape[1], xTrain.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(xTrain, yTrain, epochs=50, batch_size=72, validation_data=(xTest, yTest), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

NameError: name 'Sequential' is not defined